In [1]:
# 事实证明，研究讨论“比单个层大”但“比整个模型小”的组件更有价值。 
# 例如，在计算机视觉中广泛流行的ResNet-152架构就有数百层， 这些层是由层组（groups of layers）的重复模式组成。
# 这个ResNet架构赢得了2015年ImageNet和COCO计算机视觉比赛 的识别和检测任务
# 为了实现这些复杂的网络，我们引入了神经网络块的概念。 块（block）可以描述单个层、由多个层组成的组件或整个模型本身。
# 使用块进行抽象的一个好处是可以将一些块组合成更大的组件

In [2]:
# 从编程的角度来看，块由类（class）表示。 
# 它的任何子类都必须定义一个将其输入转换为输出的前向传播函数， 并且必须存储任何必需的参数。 
# 注意，有些块不需要任何参数。 最后，为了计算梯度，块必须具有反向传播函数。 
# 在定义我们自己的块时，由于自动微分提供了一些后端实现，我们只需要考虑前向传播函数和必需的参数。

In [3]:
# pytorch 框架名称
# torch 顶层python包
# torch.nn 神经网络的模块
# torch.nn.init 模块下的子模块,用来初始化函数,不可实例化,用来组织代码
# torch.Linear 类，需要实例化,用来生成对象

# 下面的代码生成一个网络，其中包含一个具有256个单元和ReLU激活函数的全连接隐藏层， 然后是一个具有10个隐藏单元且不带激活函数的全连接输出层

import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(
    nn.Linear(20, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)

X = torch.rand(2, 20)
net(X)

tensor([[-0.1658, -0.1765, -0.0201,  0.1095,  0.0533,  0.0063,  0.0406,  0.0994,
          0.0474, -0.0920],
        [-0.1071, -0.1639,  0.0363,  0.1519,  0.2295,  0.0920,  0.0325,  0.0291,
          0.1166, -0.1249]], grad_fn=<AddmmBackward0>)

##### 在这个例子中，我们通过实例化nn.Sequential来构建我们的模型， 层的执行顺序是作为参数传递的。

In [4]:
# 要想直观地了解块是如何工作的，最简单的方法就是自己实现一个。 在实现我们自定义块之前，我们简要总结一下每个块必须提供的基本功能
# 1.将输入数据作为其前向传播函数的参数
# 2.通过前向传播函数来生成输出。请注意，输出的形状可能与输入的形状不同。
# 3.计算其输出关于输入的梯度，可通过其反向传播函数进行访问。通常这是自动发生的。
# 4.存储和访问前向传播计算所需的参数。
# 5.根据需要初始化模型参数。

In [5]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256) # 隐藏层
        self.out = nn.Linear(256, 10) # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义
        return self.out(F.relu(self.hidden(X)))

In [6]:
net = MLP()
net(X)

tensor([[ 0.1400,  0.2154,  0.1160,  0.1263,  0.0646,  0.2333,  0.1321, -0.0260,
          0.0231,  0.1256],
        [ 0.1629,  0.3520,  0.0478,  0.1339,  0.1088,  0.2522,  0.2212, -0.0209,
          0.1679,  0.1416]], grad_fn=<AddmmBackward0>)

In [7]:
# 现在我们可以更仔细地看看Sequential类是如何工作的， 回想一下Sequential的设计是为了把其他模块串起来。 
# 为了构建我们自己的简化的MySequential， 我们只需要定义两个关键函数：
# 1.一种将块逐个追加到列表中的函数
# 2.一种前向传播函数，用于将输入按追加块的顺序传递给块组成的“链条”

In [8]:
class MySequential(nn.Module):
    # *表示收集参数，打包成一个元组
    # args 表示 arguments， 即未知参数
    # kwargs 表示 keyword arguments
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module # 把module注册子模块

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [9]:
# __init__函数将每个模块逐个添加到有序字典_modules中。 
# 读者可能会好奇为什么每个Module都有一个_modules属性？ 以及为什么我们使用它而不是自己定义一python列表？
# 简而言之，_modules的主要优点是： 在模块的参数初始化过程中， 系统知道在_modules字典中查找需要初始化参数的子块。

##### 当MySequential的前向传播函数被调用时， 每个添加的块都按照它们被添加的顺序执行。 现在可以使用我们的MySequential类重新实现多层感知机

In [10]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0004,  0.0890,  0.1061,  0.1418,  0.0141,  0.0941, -0.0100,  0.1604,
          0.0260,  0.0766],
        [ 0.0467,  0.0168, -0.0311,  0.0747, -0.0873,  0.0077,  0.0951,  0.2032,
          0.1095,  0.0817]], grad_fn=<AddmmBackward0>)

In [11]:
# Sequential类使模型构造变得简单， 允许我们组合新的架构，而不必定义自己的类。
# 然而，并不是所有的架构都是简单的顺序架构。 当需要更强的灵活性时，我们需要定义自己的块。 
# 例如，我们可能希望在前向传播函数中执行Python的控制流。 此外，我们可能希望执行任意的数学运算， 而不是简单地依赖预定义的神经网络层。

In [13]:
# 到目前为止， 我们网络中的所有操作都对网络的激活值及网络的参数起作用。
# 然而，有时我们可能希望合并既不是上一层的结果也不是可更新参数的项， 我们称之为常数参数（constant parameter）
# 比如我要计算 f(x,w) = c * w * x 的层，但是c是某个优化过程中没有更新的量
# 因此我们实现了一个FixedHiddenMLP类，如下所示
# 注意context的 X = torch.rand(2, 20)

class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)  # Linear(in_features, out_features)

    def forward(self, X):
        X  = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(X @ self.rand_weight + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [14]:
# 在上面的例子中，我们实现了一个隐藏层
# 其权重（self.rand_weight）在实例化时被随机初始化，之后为常量。 
# 这个权重不是一个模型参数，因此它永远不会被反向传播更新。 然后，神经网络将这个固定层的输出通过一个全连接层。

# 注意，在返回输出之前，模型做了一些不寻常的事情： 它运行了一个while循环，在L1范数大于1的条件下， 将输出向量除以2，直到它满足条件为止

# 注意，此操作可能不会常用于在任何实际任务中， 我们只展示如何将任意代码集成到神经网络计算的流程中。

In [16]:
net = FixedHiddenMLP()
net(X)

tensor(0.0601, grad_fn=<SumBackward0>)